In this notebook, we follow the pilot test noted in the [previous notebook](https://github.com/tingsyo/dlgridsat/blob/main/notebook/09_GLM_with_feature_vectors.ipynb), and wrap up the process with:
- [cross validation](https://scikit-learn.org/stable/modules/cross_validation.html),
- more detailed [evaluation metrics](https://en.wikipedia.org/wiki/Confusion_matrix), and
- looping through event-types and feature-vector-types.

In [1]:
import numpy as np
import pandas as pd
import os, re, logging

logging.basicConfig(level=logging.DEBUG)

# Parameters
FV_NAMES = ['PCA', 'CAE', 'CVAE', 'Pretrained-BigEarth', 'Pretrained-ImageNet']
FV_FILES = ['fv_pca.zip', 'fv_cae.zip', 'fv_cvae.zip', 'fv_ptbe.zip', 'fv_ptin.zip']
EVENT_NAMES = []
EVENT_FILE = 'tad_filtered.csv'
CV_FOLD = 10
DATAPATH = '../data/'


# Read events
def read_events(file, verbose=0):
    events = pd.read_csv(file, index_col=0)
    if verbose>0:
        print(events.shape)
        for c in events.columns:
            print(c + '\t counts: ' + str(events[c].sum()) + 
                         '\t prob:' + str(events[c].sum()/events.shape[0]))
    return(events)


# Function to give report for binary classifications
def evaluate_binary(yt, yp, id=None, ythresh=1.):
    from sklearn.metrics import confusion_matrix
    ytb = (yt>=ythresh)*1
    ypb = (yp>=ythresh)*1
    # Derive metrics
    output = {'id':id}
    TN, FP, FN, TP = confusion_matrix(ytb, ypb).ravel()
    output['true_positive'] = np.round(TP,2)
    output['false_positive'] = np.round(FP,2)
    output['false_negative'] = np.round(FN,2)
    output['true_negative'] = np.round(TN,2)
    output['sensitivity'] = np.round(TP/(TP+FN),2)
    output['specificity'] = np.round(TN/(FP+TN),2)
    output['prevalence'] = np.round((TP+FN)/(FN+TP+FP+TN),8)
    output['ppv'] = np.round(TP/(TP+FP),4)
    output['npv'] = np.round(TN/(TN+FN),4)
    output['fpr'] = np.round(FP/(FP+TN),4)
    output['threat_score'] = np.round(TP/(TP+FP+FN),4)
    output['accuracy'] = np.round((TP+TN)/(FN+TP+FP+TN),4)
    output['balanced_accuracy'] = np.round((TP/(TP+FN) + TN/(FP+TN))*0.5,4)
    output['F1'] = np.round(2*TP/(2*TP+FP+FN),4)
    output['MCC'] = np.round((TP*TN-FP*FN)/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),4)
    output['informedness'] = np.round(output['sensitivity'] + output['specificity'] - 1,4)
    output['markedness'] = np.round(output['ppv'] + output['npv'] -1,4)
    return(output)

# Test
events = read_events(DATAPATH+EVENT_FILE, verbose=1)
fv = pd.read_csv(DATAPATH+FV_FILES[0], compression='zip', index_col=0)
print(events.shape)
print(fv.shape)

(1461, 8)
CS	 counts: 12	 prob:0.008213552361396304
TYW	 counts: 65.0	 prob:0.044490075290896644
NWPTY	 counts: 702	 prob:0.4804928131416838
FT	 counts: 244	 prob:0.16700889801505817
NE	 counts: 471	 prob:0.32238193018480493
SWF	 counts: 406	 prob:0.2778918548939083
HRD	 counts: 420	 prob:0.2874743326488706
HRH	 counts: 520	 prob:0.35592060232717315
(1461, 8)
(1461, 2048)


In [2]:
# Clean up NAs
def clean_nans(x, y):
    # Merge x and y by index
    data = pd.merge(y, x, left_index=True, right_index=True)
    nrow_original = data.shape[0]
    # Drop NaNs
    data = data.dropna(axis=0, how='any')
    nrow_dropna = data.shape[0]
    # Retrieve original index as dates and reset
    dates = data.index
    data = data.reset_index(drop=True)
    print('Dropping rows containing NaNs: ' + str(nrow_original - nrow_dropna))
    y = data.iloc[:,0]
    x = data.iloc[:,1:]
    return((x, y, dates))

# Create cross validation folds
def create_cv_folds(x, y, kfold=5, randseed=123):
    from sklearn.model_selection import StratifiedKFold, KFold
    # clean up nans
    x, y, dates = clean_nans(x, y)
    # Create CV
    skf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=randseed)
    splits = skf.split(x, y)
    # 
    return((splits, x, y, dates))

# Test
splits, x, y, dates = create_cv_folds(x=fv, y=events['HRD'])

print('Class distributions (0,1) in CV folds:')
for train, test in splits:
    print('train -  {}   |   test -  {}'
          .format(np.bincount(y[train]), np.bincount(y[test])))

Dropping rows containing NaNs: 13
Class distributions (0,1) in CV folds:
train -  [826 332]   |   test -  [206  84]
train -  [825 333]   |   test -  [207  83]
train -  [825 333]   |   test -  [207  83]
train -  [826 333]   |   test -  [206  83]
train -  [826 333]   |   test -  [206  83]


Thus, we can create CV folds and iterate through them.

The package `scikit-learn` provides a faster way to perform such task with [`sklearn.model_selection.cross_val_predict`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) and [`sklearn.model_selection.cross_validate`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate). We can directly get the predictions with CV with this function, and then get evaluations.


In [18]:
# Evaluate one FV-Event pair
def evaluate_fv_event_pair(fv, event, fv_id=None, kfold=5, randseed=123):
    from sklearn.model_selection import StratifiedKFold
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import cross_val_predict, cross_validate
    # Create CV folds
    x, y, dates = clean_nans(x=fv, y=events['HRD'])
    skf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=randseed)
    clf = LogisticRegression(random_state=0, max_iter=1000)
    # 
    y_pred = cross_val_predict(clf, x, y, cv=skf)
    eval_all = evaluate_binary(y, y_pred, id=fv_id)
    print(eval_all)
    # 
    eval_cv = cross_validate(clf, x, y, cv=skf, return_train_score=True,
                             scoring=['accuracy', 'recall', 'precision', 'roc_auc', 'f1'])
    print(eval_cv)
    return(eval_all, eval_cv)


# test
eval_all, eval_cv = evaluate_fv_event_pair(fv, events['HRD'], fv_id='PCA-HRD', kfold=10)

Dropping rows containing NaNs: 13
{'id': 'PCA-HRD', 'true_positive': 156, 'false_positive': 210, 'false_negative': 260, 'true_negative': 822, 'sensitivity': 0.38, 'specificity': 0.8, 'prevalence': 0.28729282, 'ppv': 0.4262, 'npv': 0.7597, 'fpr': 0.2035, 'threat_score': 0.2492, 'accuracy': 0.6754, 'balanced_accuracy': 0.5858, 'F1': 0.399, 'MCC': 0.1786, 'informedness': 0.18, 'markedness': 0.1859}
{'fit_time': array([0.19704795, 0.2200613 , 0.22205424, 0.20004654, 0.19805074,
       0.18904567, 0.19404793, 0.20804834, 0.20904827, 0.21905065]), 'score_time': array([0.02500248, 0.02399397, 0.02400184, 0.02300429, 0.02299953,
       0.02300191, 0.02300262, 0.02300429, 0.02300525, 0.02300501]), 'test_accuracy': array([0.67586207, 0.69655172, 0.67586207, 0.69655172, 0.65517241,
       0.68275862, 0.64137931, 0.65517241, 0.66666667, 0.70833333]), 'train_accuracy': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), 'test_recall': array([0.38095238, 0.33333333, 0.42857143, 0.38095238, 0.38095238,


In [20]:
for k in eval_cv.keys():
    print(k+'_mean'+ '\t' + str(np.round(np.mean(eval_cv[k]),4)))
    print(k+'_std'+ '\t' + str(np.round(np.std(eval_cv[k]),4)))

fit_time_mean	0.2057
fit_time_std	0.0112
score_time_mean	0.0234
score_time_std	0.0007
test_accuracy_mean	0.6754
test_accuracy_std	0.0202
train_accuracy_mean	1.0
train_accuracy_std	0.0
test_recall_mean	0.3751
test_recall_std	0.0602
train_recall_mean	1.0
train_recall_std	0.0
test_precision_mean	0.427
test_precision_std	0.0404
train_precision_mean	1.0
train_precision_std	0.0
test_roc_auc_mean	0.6299
test_roc_auc_std	0.0508
train_roc_auc_mean	1.0
train_roc_auc_std	0.0
test_f1_mean	0.3972
test_f1_std	0.0455
train_f1_mean	1.0
train_f1_std	0.0


Now, we can loop through all FV-types and event-types, and collect the evaluation results. Let's beatify the output of the evaluation of single FV-Event pair.

In [24]:
# Evaluate one FV-Event pair
def evaluate_fv_event_pair(fv, event, fv_id=None, kfold=5, randseed=123):
    from sklearn.model_selection import StratifiedKFold
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import cross_val_predict, cross_validate
    # Create CV folds
    x, y, dates = clean_nans(x=fv, y=event)
    skf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=randseed)
    clf = LogisticRegression(random_state=0, max_iter=10000)
    # Get predictions from CV
    y_pred = cross_val_predict(clf, x, y, cv=skf)
    eval_all = evaluate_binary(y, y_pred, id=fv_id)
    #print(eval_all)
    # Get evaluations for each CV fold
    eval_cv = cross_validate(clf, x, y, cv=skf, return_train_score=True,
                             scoring=['accuracy', 'recall', 'precision', 'roc_auc', 'f1'])
    for k in eval_cv.keys():
        eval_all[k+'_mean'] = np.round(np.mean(eval_cv[k]),4)
        eval_all[k+'_std'] = np.round(np.std(eval_cv[k]),4)
    #print(eval_cv)
    return(eval_all, eval_cv)


# test
eval_all, eval_cv = evaluate_fv_event_pair(fv, events['HRD'], fv_id='PCA-HRD', kfold=10)
print(eval_all)

Dropping rows containing NaNs: 13
{'id': 'PCA-HRD', 'true_positive': 156, 'false_positive': 234, 'false_negative': 260, 'true_negative': 798, 'sensitivity': 0.38, 'specificity': 0.77, 'prevalence': 0.28729282, 'ppv': 0.4, 'npv': 0.7543, 'fpr': 0.2267, 'threat_score': 0.24, 'accuracy': 0.6588, 'balanced_accuracy': 0.5741, 'F1': 0.3871, 'MCC': 0.1512, 'informedness': 0.15, 'markedness': 0.1543, 'fit_time_mean': 1.0502, 'fit_time_std': 0.1501, 'score_time_mean': 0.0239, 'score_time_std': 0.0014, 'test_accuracy_mean': 0.6589, 'test_accuracy_std': 0.032, 'train_accuracy_mean': 0.9936, 'train_accuracy_std': 0.0017, 'test_recall_mean': 0.3745, 'test_recall_std': 0.0967, 'train_recall_mean': 0.9816, 'train_recall_std': 0.0045, 'test_precision_mean': 0.3952, 'test_precision_std': 0.0705, 'train_precision_mean': 0.9959, 'train_precision_std': 0.0018, 'test_roc_auc_mean': 0.6336, 'test_roc_auc_std': 0.0496, 'train_roc_auc_mean': 0.9996, 'train_roc_auc_std': 0.0002, 'test_f1_mean': 0.3832, 'test_f

In [25]:
# Parameters
FV_NAMES = ['PCA', 'CAE', 'CVAE', 'Pretrained-BigEarth', 'Pretrained-ImageNet']
FV_FILES = ['fv_pca.zip', 'fv_cae.zip', 'fv_cvae.zip', 'fv_ptbe.zip', 'fv_ptin.zip']
EVENT_NAMES = ['CS', 'TYW', 'NWPTY', 'FT', 'NE', 'SWF', 'HRD', 'HRH']
EVENT_FILE = 'tad_filtered.csv'
CV_FOLD = 10
DATAPATH = '../data/'

events = read_events(DATAPATH+EVENT_FILE, verbose=1)
results = []
cv_details = {}

# Loop
for i in range(len(FV_NAMES)):
    for j in range(len(EVENT_NAMES)):
        fv = pd.read_csv(DATAPATH+FV_FILES[i], compression='zip', index_col=0)
        fv_name = FV_NAMES[i]
        event_name = EVENT_NAMES[j]
        exp_id = fv_name+'-'+event_name
        print(exp_id)
        eval_all, eval_cv = evaluate_fv_event_pair(fv, events[event_name], fv_id=exp_id, kfold=10)
        results.append(eval_all)
        cv_details[exp_id] = eval_cv



(1461, 8)
CS	 counts: 12	 prob:0.008213552361396304
TYW	 counts: 65.0	 prob:0.044490075290896644
NWPTY	 counts: 702	 prob:0.4804928131416838
FT	 counts: 244	 prob:0.16700889801505817
NE	 counts: 471	 prob:0.32238193018480493
SWF	 counts: 406	 prob:0.2778918548939083
HRD	 counts: 420	 prob:0.2874743326488706
HRH	 counts: 520	 prob:0.35592060232717315
PCA-CS
Dropping rows containing NaNs: 13


C:\Users\tsyo\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tsyo\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tsyo\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tsyo\AppData\Local\Continuum\anaconda3\l

PCA-TYW
Dropping rows containing NaNs: 13
PCA-NWPTY
Dropping rows containing NaNs: 13
PCA-FT
Dropping rows containing NaNs: 13
PCA-NE
Dropping rows containing NaNs: 13
PCA-SWF
Dropping rows containing NaNs: 13
PCA-HRD
Dropping rows containing NaNs: 13
PCA-HRH
Dropping rows containing NaNs: 13
CAE-CS
Dropping rows containing NaNs: 13


C:\Users\tsyo\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tsyo\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tsyo\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tsyo\AppData\Local\Continuum\anaconda3\l

CAE-TYW
Dropping rows containing NaNs: 13
CAE-NWPTY
Dropping rows containing NaNs: 13
CAE-FT
Dropping rows containing NaNs: 13
CAE-NE
Dropping rows containing NaNs: 13
CAE-SWF
Dropping rows containing NaNs: 13
CAE-HRD
Dropping rows containing NaNs: 13
CAE-HRH
Dropping rows containing NaNs: 13
CVAE-CS
Dropping rows containing NaNs: 13


...

Pretrained-BigEarth-TYW
Dropping rows containing NaNs: 13


C:\Users\tsyo\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tsyo\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tsyo\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Pretrained-BigEarth-NWPTY
Dropping rows containing NaNs: 13
Pretrained-BigEarth-FT
Dropping rows containing NaNs: 13
Pretrained-BigEarth-NE
Dropping rows containing NaNs: 13
Pretrained-BigEarth-SWF
Dropping rows containing NaNs: 13
Pretrained-BigEarth-HRD
Dropping rows containing NaNs: 13
Pretrained-BigEarth-HRH
Dropping rows containing NaNs: 13
Pretrained-ImageNet-CS
Dropping rows containing NaNs: 13


C:\Users\tsyo\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tsyo\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tsyo\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tsyo\AppData\Local\Continuum\anaconda3\l

Pretrained-ImageNet-TYW
Dropping rows containing NaNs: 13
Pretrained-ImageNet-NWPTY
Dropping rows containing NaNs: 13
Pretrained-ImageNet-FT
Dropping rows containing NaNs: 13
Pretrained-ImageNet-NE
Dropping rows containing NaNs: 13
Pretrained-ImageNet-SWF
Dropping rows containing NaNs: 13
Pretrained-ImageNet-HRD
Dropping rows containing NaNs: 13
Pretrained-ImageNet-HRH
Dropping rows containing NaNs: 13


In [27]:
pd.DataFrame(results).to_csv('../data/exp_results.csv', index=False)